In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#predictions from Lag-LLama model
pred_df = pd.read_pickle('/content/drive/My Drive/lag_llama_elec_BE_C3w_P24h_pred_interval.pkl')
pred_df = pred_df[['time', 'actual', 'step1']]
pred_df.rename(columns = {'step1':'pred'}, inplace = True)

In [ ]:
pred_df

,time,actual,pred
0,2016-11-12 00:00:00,59.090000,"[57.3092, 42.5022, 26.912748, 54.28474, 36.948..."
1,2016-11-12 01:00:00,56.259998,"[21.06731, 52.11542, 62.83943, 34.306038, 58.6..."
2,2016-11-12 02:00:00,49.700001,"[30.19452, 46.344624, 47.319748, 40.79335, 36...."
3,2016-11-12 03:00:00,46.470001,"[53.430977, 54.377594, 13.115349, 47.69988, 36..."
4,2016-11-12 04:00:00,43.799999,"[41.21124, 36.518837, 70.96763, 30.347853, 30...."
...,...,...,...
1171,2016-12-30 19:00:00,57.419998,"[64.482635, 73.33732, 57.583015, 50.79124, 54...."
1172,2016-12-30 20:00:00,57.560001,"[53.932278, 45.52086, 53.712296, 42.599564, 47..."
1173,2016-12-30 21:00:00,52.180000,"[43.978916, 47.92664, 41.893837, 42.14767, 43...."
1174,2016-12-30 22:00:00,54.380001,"[62.323307, 49.840366, 44.8881, 46.509407, 42...."


In [ ]:
def bootstrap_mean(data, size):
  """Calculates the mean of a bootstrap sample."""
  bootstrap_sample = np.random.choice(data, size, replace=True)
  return np.mean(bootstrap_sample)

# Create 20 bootstrap mean columns
for i in range(100):
  pred_df[f'bootstrap_mean_{i+1}'] = pred_df['pred'].apply(lambda x: bootstrap_mean(x, 40))


# Calculate the mean of the 'pred' column
pred_df['point_pred'] = pred_df['pred'].apply(lambda x: bootstrap_mean(x, 1))

bootstrap_mean_cols = [col for col in pred_df.columns if 'bootstrap_mean_' in col]
pred_df['bootstrap_mean_all'] = pred_df[bootstrap_mean_cols].mean(axis=1)

<ipython-input-5-70d06fc0dc97>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pred_df[f'bootstrap_mean_{i+1}'] = pred_df['pred'].apply(lambda x: bootstrap_mean(x, 40))
<ipython-input-5-70d06fc0dc97>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pred_df[f'bootstrap_mean_{i+1}'] = pred_df['pred'].apply(lambda x: bootstrap_mean(x, 40))
<ipython-input-5-70d06fc0dc97>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join

In [ ]:
# prompt: calculate MSE for pred_mean and pred_bootstrap_all with respect to actual

from sklearn.metrics import mean_squared_error

# Calculate MSE for pred_mean
mse_mean = mean_squared_error(pred_df['actual'], pred_df['point_pred'])
print(f"MSE for point_pred: {mse_mean}")

# Calculate MSE for pred_bootstrap_all
mse_bootstrap_all = mean_squared_error(pred_df['actual'], pred_df['bootstrap_mean_all'])
print(f"MSE for pred_bootstrap_all: {mse_bootstrap_all}")


MSE for point_pred: 1722.0899658203125
MSE for pred_bootstrap_all: 795.648681640625
